## River forecasting

## Setup packages
Note - Operational version available at: https://mybinder.org/v2/gh/scottmreed/GaugeGeek/main 

In [ ]:
import os
import datetime
import pytest
import requests

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import scipy as sp
from pandas import concat

from datetime import datetime

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.sequence import TimeseriesGenerator

import sklearn.metrics as metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import median_absolute_error

from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.utils import Bunch
from sklearn.compose import make_column_transformer

from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor 
from sklearn.svm import SVC

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

IPython.display.clear_output()
%config InlineBackend.figure_format = 'retina'

In [ ]:
# change directory if needed
%cd ./datasets

## Check the weather 

In [ ]:
# You can upload csv data that NOAA sends you by email from https://www.ncdc.noaa.gov/cdo-web/

dataset = pd.read_csv('DenverWeatherHistory.csv', index_col=2)
remove = ['TOBS', 'STATION','NAME']
weather_df = dataset[dataset.columns.difference(remove)]
dataset.index.name = 'Date'
print(weather_df.describe())

# if you want weather for a different zipcode or date, you can use the code below
# date_time = pd.to_datetime(weather_df.pop('DATE'), format='%d.%m.%Y')
# timestamp_s = date_time.map(datetime.datetime.timestamp)

# or download directly from NOAA API
# from noaa_sdk import NOAA
# n = NOAA()
# observations = n.get_observations('80204','US',start='2021-02-11', end='2021-02-12')
# for observation in observations:
#     temp = (observation['temperature']['value'])
#     time_stamp = (observation['timestamp'])
#     print(time_stamp, temp)

# Load river data

In [ ]:
# load water flow data 
# add your token if you have an account with Denver Water and then you can download more data per day
token = '####'
# reduce the page size if you are concerned about running out of requests
page_Size = 50000

def Load_gauge(gauge_label, gauge_name, start_date, end_date):
    flows=[]
    dates=[]

    URL = f'https://dwr.state.co.us/Rest/GET/api/v2/surfacewater/surfacewatertsday/?format=json&dateFormat=spaceSepToSeconds&fields=measDate%2Cvalue&abbrev={gauge_name}&min-measDate={start_date}&max-measDate={end_date}&pageSize={page_Size}'
    urlData = requests.get(URL).content
    rawData = pd.read_json(urlData.decode('utf-8'))
    data=(rawData.ResultList)

    for count,entry in enumerate(data):
        dates.append(data.iloc[count]['measDate'])
        flows.append(data.iloc[count]['value'])
        element = data.iloc[count]

    date_series = pd.Series(dates) 
    flow_series = pd.Series(flows) 

    frame = {'Date': date_series, 'cfs_'+gauge_label: flow_series} 
    Gauge_Data = pd.DataFrame(frame) 

    Gauge_Data['Date'] = pd.to_datetime(Gauge_Data['Date'])
    Gauge_Data = Gauge_Data.set_index('Date')
    
    return Gauge_Data

name_1 = 'roberts' # red
name_2 = 'deckers' # blue
name_3 = 'grant' # green
start_date = '01%2F01%2F2010' # 01%2F01%2F2010 means January 1, 2010
end_date = '1%2F31%2F2021'
begin_date = start_date.replace('%2F','-')
until_date = end_date.replace('%2F','-')
gauge_1 = Load_gauge(name_1,'ROBTUNCO', start_date, end_date)
gauge_2 = Load_gauge(name_2,'PLASPLCO', start_date, end_date)
gauge_3 = Load_gauge(name_3,'PLAGRACO', start_date, end_date)

# You can test out other API calls here: https://dwr.state.co.us/Rest/GET/Help/SurfaceWaterTSDayGenerator

In [ ]:
# print seasonal view as overlay to preview data

def plot_preview(gauge1, gauge2, gauge3, start, stop):
    plt.figure(figsize=(14, 4))
    plt.title('gauges')
    plot_key = 'red is gauge 1, blue is gauge 2, green is gauge 3'
    plt.text(gauge_1.index[0],max(gauge_2.values),plot_key)
    plt.plot(gauge1[start:stop], color='red')
    plt.plot(gauge2[start:stop], color='blue')
    plt.plot(gauge3[start:stop], color='green')

plot_preview(gauge_1,gauge_2,gauge_3, 0, len(gauge_1))

In [ ]:
from IPython.display import Image
# %cd datasets
Image("./plattecamp.jpg")

## Combine river gauges

## Merge River data and combine with weather data

In [ ]:
def day_shift(data, col_names, days_in=1, days_out=1):
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    for i in range(days_in, 0, -1):
        cols.append(df.shift(i))
        names += [(f'{col_names[j]}%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, days_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [(f'{col_names[j]}') for j in range(n_vars)]
        else:
            names += [(f'{col_names[j]}(t+%d)' % (i)) for j in range(n_vars)]
    agg = concat(cols, axis=1) 
    agg.columns = names
    return agg

combined_gauges = pd.concat([gauge_1, gauge_2, gauge_3], axis=1)
combined_gauges_index = combined_gauges.index
col_names = combined_gauges.columns.values.tolist()
values = combined_gauges.values

# days_out is number of shifts
combo_gauge = day_shift(values, col_names, 0, 3)
combo_gauge.index = combined_gauges_index
print(combo_gauge)

In [ ]:
# Add weather to array
wet_gauges = weather_df.join(combo_gauge, how='outer')
wet_gauges = wet_gauges.dropna()
wet_gauges.head()

In [ ]:
# Age the weather data by 1 day. This stands in as what the weather forecast probably would have been
# For example if the high temp on Sat is 60 the forecasted high temp on Fri for Sat was prob ~60
# Need to loop, functionalize, and add more days
days_back = 2
wet_gauges.loc[:,f'Tmax(t+{days_back})'] = wet_gauges.loc[:,'TMAX'].shift(-days_back)
wet_gauges.loc[:,f'Tmin(t+{days_back})'] = wet_gauges.loc[:,'TMIN'].shift(-days_back)
# wet_gauges = wet_gauges.dropna()

wet_gauges.head()

## Parse seasons

In [ ]:
# split the data into 3 'seasons' and pick one to move forward with
# Different models might work better during different seasons.
# The number is the month number so spring is 5 to 6 or May and June.
wet_gauges_spring = wet_gauges[wet_gauges.index.month > 4]
wet_gauges_spring = wet_gauges_spring[wet_gauges_spring.index.month < 7]
wet_gauges_summer = wet_gauges[wet_gauges.index.month > 6]
wet_gauges_summer = wet_gauges_summer[wet_gauges_summer.index.month < 9]
wet_gauges_fall = wet_gauges[wet_gauges.index.month >8]
wet_gauges_fall = wet_gauges_fall[wet_gauges_fall.index.month < 11]

wet_gauges_fall.describe()
season_selected = wet_gauges_fall
season = ('fall')

## Take a look at the data

## A Heatmap shows correlatations between each variable 
The diagonal is fully correlated because that is a self-correlation.

In [ ]:
# This shows how correalted two inputs are.
# It can preview whether an input you selected is providing unique information to the model. 
def show_heatmap(data):
    plt.matshow(data.corr())
#     plt.figure(5,5)
    plt.xticks(range(data.shape[1]), data.columns, fontsize=14, rotation=90)
    plt.gca().xaxis.tick_bottom()
    plt.yticks(range(data.shape[1]), data.columns, fontsize=14)

    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title("Feature Correlation Heatmap", fontsize=14)
    plt.show()

show_heatmap(season_selected)

In [ ]:
# Get into the weeds and create a grid of Axes such that each variable is shared 
# across the y-axes across a single row and the x-axes across a single column. 
# The diagonal plots are  a univariate distribution plots by histogram 
# showing the range of values for that given variable.
# The axes labels may be hard to read but they match the heatmap above
# This is slow and can be skipped.

_ = sns.pairplot(season_selected, diag_kind = 'hist')
# could bootstrap out outliers

In [ ]:
# a better method for viewing the distribution of values for each variable as shown on the diagonal above
season_selected_mean = season_selected.mean()
season_selected_std = season_selected.std()
df_std = (season_selected - season_selected_mean) / season_selected_std
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(season_selected.keys(), rotation=90)

## Normalize data

### Create a "bunch" of the data.

In [ ]:
# Bunch the data
# isolate the 'cfs' column you want to predict as target
predict = name_3
print(f'this is the gauge being used for forecast: cfs_{predict}')
col_names = season_selected.columns.tolist()
col_names.remove(f'cfs_{predict}')

test_df2_combo = season_selected
test_df2_nonflow = season_selected[col_names].copy()

test_df2_flow = test_df2_combo[f'cfs_{predict}']

test_df2_nonflow.index = pd.RangeIndex(len(test_df2_nonflow.index))
test_df2_flow.index = pd.RangeIndex(len(test_df2_flow.index))

bunched_data = Bunch(data=test_df2_nonflow, target=test_df2_flow)

X = bunched_data.data
y = bunched_data.target.values.ravel()

### Split into test and train

In [ ]:
# The training set is applied to train the model, finding
# the optimal weights, or coefficients, for linear regression.
# The test set is used for an unbiased evaluation of the final model. 

# Defaults to 75% train and 25% test
# Could parse it out by years or seasons
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=None, shuffle=False
)

rows = (test_df2_combo.index)
n = (len(rows))
test_rows = rows[int(n*.75):]

## Time to start some machine learning

In [ ]:
# Lasso is a linear regression technique. It assumes a linear relationship between the
# output variable, river flow in cfs, and each of the input variables.
# The model it develops applies a penalty to including variables that
# do not help the model, going so far as to eliminate variables. This way if we added a variable
# that doesn't help the model it is discarded.

# this blog does a really good job of describing this and other machine learning approaches: https://machinelearningmastery.com

preprocessor1 = make_column_transformer(
        (StandardScaler(), ['TMAX']),
        remainder=StandardScaler(),verbose=True)

model2 = make_pipeline(
    preprocessor1,
    TransformedTargetRegressor(
        regressor=LassoCV(alphas=np.logspace(-10, 10, 21), max_iter=100000),
    ),)

_ = model2.fit(X_train, y_train)

y_pred = model2.predict(X_train)
mae = median_absolute_error(y_train, y_pred)
string_score = f'Mean Avg Error on training set: {mae:.2f} cfs'
y_pred = model2.predict(X_test)
mae = median_absolute_error(y_test, y_pred)
string_score += f'\nMean Avg Error on testing set: {mae:.2f} cfs'

fig, ax = plt.subplots(figsize=(7, 3))
plt.scatter(y_test, y_pred)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c="red")

plt.text(20,400,string_score)

plt.title(f'Predictions for S. Platte at:{predict} in {season} tested on: {begin_date} to {until_date}')
plt.ylabel('Predicted flow [cfs]')
plt.xlabel('Real Flow [cfs]')

In [ ]:
# These are the weighting for the model. 
# They show how each variable in the model predicted flow at the gauge you are predicting. A large bar to the right means that
# if there was a large increase in that variable on that day there would be a large increase in flow at the site you are predicting. 
coefs = pd.DataFrame(
    model2.named_steps['transformedtargetregressor'].regressor_.coef_,
    columns=['Coefficients'], index=bunched_data.data.columns
)
coefs.plot(kind='barh', figsize=(9, 7))
string_score = f'Weightings for {predict} in {season} model on : {begin_date} to {until_date}'
string_score += f'\nMean Avg Error on test data: {mae:.2f} cfs'
plt.title(string_score)
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)

In [ ]:
# Here is a simple time series plot
# Change start and stop to plot other time periods
# Remember there are diconotinuities in the data

plt.title('Comparison of real(blue) to predicted(red)')
plt.figure(figsize=(14, 4))
plot_key = 'blue is real, red is predicted'
plt.title('Comparison of real to predicted')
start = 0
stop = 40
plt.scatter(test_rows[start:stop],y_pred[start:stop], color = 'red')
plt.plot(test_rows[start:stop],y_test[start:stop])
